# Equalizer WFTM-MPX LP

In [1]:
import json
import pandas as pd
from chaino.scheduler.call import parse_address

mpx_erc20_address = "0x66eEd5FF1701E6ed8470DC391F05e27B1d0657eb"
mlp_erc20_address = "0xd5c313DE2d33bf36014e6c659F13acE112B80a8E"
vault_address = "0x3CB54f0eB62C371065D739A34a775CC16f46563e"
mlp_manager_address = "0xA3Ea99f8aE06bA0d9A6Cf7618d06AEa4564340E9"
musd_erc20_address = "0xB7209EbCBF71c0ffA1585B4468A11CFfdcDBB9a9"
esmpx_erc20_address = "0xe0f606e6730bE531EeAf42348dE43C2feeD43505"
reward_router_address = "0x20De7f8283D377fA84575A26c9D484Ee40f55877"
reward_reader_address = "0x512F8D4E28EB53A6d036aEDA9C5a4D1De6DBD543"
weth_erc20_address = "0x21be370D5312f44cB42ce377BC9b8a0cEF1A4C83"
tracker_staked_mlp_address = "0xa4157E273D88ff16B3d8Df68894e1fd809DbC007" # staked MPX
tracker_staked_mpx_address = "0x49A97680938B4F1f73816d1B70C3Ab801FAd124B" # fee + staked MLP
equalizer_wftm_mpx_address = "0xdE26e98d868FE02fFfb6DF26E638995124d3Ca13"
equalizer_wftm_mpx_address_new = "0xF8eed2665FD11a8431fc41b2582fD5E72a1606f0"
equalizer_gauge_1 = "0x27F7cf5e918311AAF5E7185b5BcDAc158dFacf53" # 
equalizer_gauge_2 = "0x7778a0B4688321c4E705d4e9F1A072f6F1579Bf8" # equalizer
fvm_wftm_mpx_address = "0xF8eed2665FD11a8431fc41b2582fD5E72a1606f0"
fvm_wftm_mpx_gauge = "0xF89f367E0225fE68c7c28Fad0BaDc7f569987cFe"

First we obtain the MPX balance for the Equalizer LP.
These balances are queried from the MPX ERC-20 contract.

In [2]:
mpx_erc20_balances = pd.read_csv("../products/fantom-mpx-balances.csv")
pool_mpx_total = mpx_erc20_balances.query(f'address == "{equalizer_wftm_mpx_address}"')
pool_mpx_total = int(list(pool_mpx_total['balance'])[0])
pool_mpx_total / 1e18

1281894.1184612277

Then we obtain the balances of the Equalizer WFTM-MPX LP.
This represents the distribution of MPX provided for liquidity.

In [3]:
with open("../data/fantom-equalizer-wftm-mpx.json", "r") as f:
    equalizer_call_results = json.load(f)
len(equalizer_call_results)

4510

In [4]:
equalizer_lp_balances = {}
for key, value in equalizer_call_results.items():
    if value > 0:
        equalizer_lp_balances[parse_address(key)] = value

pool_lp_total = sum(equalizer_lp_balances.values())
pool_lp_total / 1e18

851396.9011039473

## Equalizer WFTM-MPX Gauges

Most of the LP is controlled by a gauge.
So, we must examine gauge contract balances to find the actual addresses that control the MPX.

In [5]:
with open("../data/fantom-equalizer-gauge-1.json", "r") as f:
    equalizer_gauge_1_call_results = json.load(f)
len(equalizer_gauge_1_call_results)

4510

In [6]:
equalizer_gauge_1_balances = {}
for key, value in equalizer_gauge_1_call_results.items():
    if value > 0:
        equalizer_gauge_1_balances[parse_address(key)] = value

gauge_1_lp_total = sum(equalizer_gauge_1_balances.values())
sum(equalizer_gauge_1_balances.values()) / 1e18

524974.0046707705

In [7]:
with open("../data/fantom-equalizer-gauge-2.json", "r") as f:
    equalizer_gauge_2_call_results = json.load(f)
len(equalizer_gauge_2_call_results)

4510

In [8]:
equalizer_gauge_2_balances = {}
for key, value in equalizer_gauge_2_call_results.items():
    if value > 0:
        equalizer_gauge_2_balances[parse_address(key)] = value

gauge_2_lp_total = sum(equalizer_gauge_2_balances.values())
sum(equalizer_gauge_2_balances.values()) / 1e18

11149.893598054376

## Attribute MPX to LP and Gauge

Finally, we attribute MPX to original addresses.
The MPX controlled by each address is held in proportion to the LP tokens controlled by that address.

Step 1. Examine the LP and directly assign LP to non-gauge addresses.

In [9]:
equalizer_lp_attributed_balances = {}
for key, value in equalizer_call_results.items():
    if value > 0 and parse_address(key) not in [equalizer_gauge_1, equalizer_gauge_2]:
        equalizer_lp_attributed_balances[parse_address(key)] = value
sum(equalizer_lp_attributed_balances.values()) / 1e18

1123.1288125757815

Step 2. Examine Gauges and assign LP to addresses.
First determine how much LP is controlled by a gauge, then assign LP based on proportion of gauge.

In [10]:
lp_controlled_by_gauge_1 = equalizer_lp_balances[equalizer_gauge_1]
lp_controlled_by_gauge_1 / 1e18

839123.0124607284

In [11]:
for key, value in equalizer_gauge_1_call_results.items():
    if value > 0 and parse_address(key) not in [equalizer_gauge_1, equalizer_gauge_2]:
        equalizer_lp_attributed_balances[parse_address(key)] = (value / gauge_1_lp_total) * lp_controlled_by_gauge_1
sum(equalizer_lp_attributed_balances.values()) / 1e18

840188.8691212935

In [12]:
lp_controlled_by_gauge_2 = equalizer_lp_balances[equalizer_gauge_2]
lp_controlled_by_gauge_2 / 1e18

11150.759830643088

In [13]:
for key, value in equalizer_gauge_2_call_results.items():
    if value > 0 and parse_address(key) not in [equalizer_gauge_1, equalizer_gauge_2]:
        equalizer_lp_attributed_balances[parse_address(key)] = (value / gauge_2_lp_total) * lp_controlled_by_gauge_2
sum(equalizer_lp_attributed_balances.values()) / 1e18

848706.3446693355

In [14]:
(1 - ((pool_lp_total - sum(equalizer_lp_balances.values())) / pool_lp_total)) * 100

100.0

We have attributed all of the LP.

## Final export of balances

In [15]:
df = pd.DataFrame.from_dict(equalizer_lp_attributed_balances, orient="index", columns=["lp_balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "lp_balance"]]
df['mpx_balance'] = (df['lp_balance'] / pool_lp_total) * pool_mpx_total
df['lp_balance'] = df['lp_balance'].apply(lambda x: int(x)).values.tolist()
df['mpx_balance'] = df['mpx_balance'].apply(lambda x: int(x)).values.tolist()
df.to_csv("../products/fantom-equalizer-balances.csv", index=False, float_format='%g')
sum(df['lp_balance']) / 1e18

848706.3446693354